### PizzaBot Order Agent

Second version of pizza order that applies a custom agent.

Adapted from https://python.langchain.com/docs/modules/agents/how_to/sharedmemory_for_tools

In [1]:
# open ai authentication

import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.getenv('OPENAI_API_KEY')

In [2]:
# email util

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from langchain.agents import tool

def get_mail_server(server='smtp.gmail.com', port=587):
    '''Allocate mail server on port'''
    mailServer = smtplib.SMTP(server, port)
    test = mailServer.starttls()  # start TLS
    mailServer.login(os.environ['GMAIL_ADDRESS'], os.environ['GMAIL_PS'])
    assert test[1].lower().find(b'ready') != -1
    return mailServer

def format_message(message, sender, receiver, subject):
    '''
    Format arguments into MIMEMultipart object
    '''

    # message
    msg = MIMEMultipart()
    msg['From'] = sender
    msg['To'] = ', '.join(receiver)
    msg['Subject'] = subject
    msg.attach(MIMEText(message, 'html'))

    return msg

#def send_message(order_summary, email):
def send_message(input):
    """
    Send the order summary (from the order summary tool)
    to the email indicated by the user
    """

    order_summary, email = input.split('|')
    
    sender = 'OrderBotPizza.com'
    subject = 'Your OrderBot Pizza Order'
    verbose = True

    msg = format_message(order_summary, sender, [email], subject)
    
    if verbose:
        print(f'sending mail from {sender} to {email}')
        print(msg)
    with get_mail_server() as mailServer:
        mailServer.sendmail(sender, email, msg.as_string())
        
    return msg, 'mail sent successfully'

In [3]:
# example use of send_message
#send_message('email test|tweinzirl@gmail.com')

# Building the Langchain Agent

!!!! Update this: summary tool has access to the conversation history

Use the following components:
 - [Conversational agent](https://python.langchain.com/en/latest/modules/agents/agents/examples/chat_conversation_agent.html) AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION
 - Conversation buffer (ConversationBufferMemory)
 - OpenAI Large Language Model (gpt-3.5-turbo)
 - Tools for menu lookup (SQLDatabaseSequentialChain), summarizing an order (custom function), and sending an email (custom function)

In [4]:
from langchain.agents import ConversationalAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.chains import SQLDatabaseChain, SQLDatabaseSequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase

from IPython.display import display, Markdown

In [5]:
# llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0301", temperature=0, verbose=True)

In [6]:
# memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
readonlymemory = ReadOnlySharedMemory(memory=memory)

In [7]:
# order summary tool from LLMChain

template = """This is a conversation between a human and a pizza order bot:

{chat_history}

Write a summary of the pizza order. Structure the summary as a table showing
the item name and the item price. Place the total price on the last line. Below
is an example for an order with two items.

Order Summary: 
<br>Item_1 Price_1 
<br>Item_2 Price_2 
<br>Total: Total price
Your reponse here:
"""

prompt = PromptTemplate(input_variables=["chat_history"], template=template)
summary_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=readonlymemory,  # use the read-only memory to prevent the tool from modifying the memory
)

In [8]:
print(prompt.template)

This is a conversation between a human and a pizza order bot:

{chat_history}

Write a summary of the pizza order. Structure the summary as a table showing
the item name and the item price. Place the total price on the last line. Below
is an example for an order with two items.

Order Summary: 
<br>Item_1 Price_1 
<br>Item_2 Price_2 
<br>Total: Total price
Your reponse here:



In [9]:
# how to test the summary chain
print(summary_chain.run({}))



> Entering new  chain...
Prompt after formatting:
This is a conversation between a human and a pizza order bot:

[]

Write a summary of the pizza order. Structure the summary as a table showing
the item name and the item price. Place the total price on the last line. Below
is an example for an order with two items.

Order Summary: 
<br>Item_1 Price_1 
<br>Item_2 Price_2 
<br>Total: Total price
Your reponse here:


> Finished chain.
Order Summary:
<br>Large Pepperoni Pizza $15.99
<br>Garlic Knots $4.99
<br>Total: $20.98


In [10]:
# sql chain: SQLDatabaseSequentialChain
db = SQLDatabase.from_uri("sqlite:///./pizza_v2.db")
sql_chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)  # used default prompt in the end

In [11]:
# agent tools
tools = [
    Tool(
        name = "Menu Search",
        func=sql_chain.run,
        description="""Call this function when asked about the menu or when adding a new item to the order.
        Be sure to return the cost of the item."""
    ),

        Tool(
        name="Order Summary",
        func=lambda x: summary_chain.run({}),
        return_direct=True,
        description="""needed to make a summary of the order, but you want to make sure it's formatted correctly. 
    Input is your thought and self-critique and output is the processed thought""",
    ),

        Tool(
        name = "Send Email",
        func=send_message,
        description="""The input to this tool should be a pipe separated list of two strings representing the
        order summary and the recipient email. For example, `large cheese pizza for $10|user@user.com` would be the input
        if you wanted the email sent to user@user.com."""
    ),
]

In [12]:
# build the agent prompt

prefix = """You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment. \
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \

You have access to the following tools:
"""

suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

agent_prompt = ConversationalAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)

In [13]:
# print agent_prompt
print(agent_prompt.template)

You are OrderBot, an automated service to collect orders for a pizza restaurant. You first greet the customer, then collects the order, and then asks if it's a pickup or delivery. You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. Finally you collect the payment. Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu.You respond in a short, very conversational friendly style. 
You have access to the following tools:


> Menu Search: Call this function when asked about the menu or when adding a new item to the order.
        Be sure to return the cost of the item.
> Order Summary: needed to make a summary of the order, but you want to make sure it's formatted correctly. 
    Input is your thought and self-critique and output is the processed thought
> Send Email: The input to this tool should be a pipe separated list of two strin

In [14]:
# constuct the llmChain with the agent prompt, and then the agent itself
llm_chain = LLMChain(llm=llm, prompt=agent_prompt)
agent = ConversationalAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)
agent_chain.handle_parsing_errors = True  # handle parsing errors from invalid/incomplete responses

### Test the Agent

In [15]:
agent_chain.run(input="hi, i am Tim and my email is tweinzirl@gmail.com")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Hi Tim! Welcome to our pizza restaurant. How can I assist you today?

> Finished chain.


'Hi Tim! Welcome to our pizza restaurant. How can I assist you today?'

In [16]:
agent_chain.run(input="What is my name and email")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Your name is Tim and your email is tweinzirl@gmail.com. How can I assist you with your order today?

> Finished chain.


'Your name is Tim and your email is tweinzirl@gmail.com. How can I assist you with your order today?'

In [17]:
agent_chain.run(input="What is your most popular pizza?")



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Menu Search
Action Input: "most popular pizza"

> Entering new  chain...


/home/timw/venv/llm/lib/python3.11/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Table names to use:
['pizzas', 'toppings']

> Entering new  chain...
most popular pizza
SQLQuery:SELECT Item, Description FROM pizzas ORDER BY Price_Large DESC LIMIT 1
SQLResult: [('Pepperoni pizza', 'Our most popular')]
Answer:Pepperoni pizza is the most popular pizza according to the description in the pizzas table.
> Finished chain.

> Finished chain.

Observation: Pepperoni pizza is the most popular pizza according to the description in the pizzas table.
Thought:Do I need to use a tool? No
AI: Our most popular pizza is the Pepperoni pizza. Would you like to order one?

> Finished chain.


'Our most popular pizza is the Pepperoni pizza. Would you like to order one?'

In [18]:
agent_chain.run("""I want the large pepperoni pizza plus sausage and extra cheese as additional toppings.
How much is this?""")



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Menu Search
Action Input: large pepperoni pizza, sausage, extra cheese

> Entering new  chain...
Table names to use:
['pizzas', 'toppings']

> Entering new  chain...
large pepperoni pizza, sausage, extra cheese
SQLQuery:SELECT pizzas.Item, pizzas.Price_Large, toppings.Price
FROM pizzas, toppings
WHERE pizzas.Item = 'Pepperoni pizza' AND toppings.Item IN ('Sausage', 'Extra cheese')
ORDER BY toppings.Price DESC
LIMIT 5
SQLResult: [('Pepperoni pizza', '12.95', '3.00'), ('Pepperoni pizza', '12.95', '2.00')]
Answer:A large pepperoni pizza with sausage and extra cheese costs $17.95.
> Finished chain.

> Finished chain.

Observation: A large pepperoni pizza with sausage and extra cheese costs $17.95.
Thought:Do I need to use a tool? No
AI: Great choice, Tim! A large pepperoni pizza with sausage and extra cheese costs $17.95. Is there anything else you would like to add to your order?

> Finished chain.


'Great choice, Tim! A large pepperoni pizza with sausage and extra cheese costs $17.95. Is there anything else you would like to add to your order?'

In [19]:
agent_chain.run('I need a drink. One large bottled water please.')



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Menu Search
Action Input: bottled water

> Entering new  chain...
Table names to use:
['drinks']

> Entering new  chain...
bottled water
SQLQuery:SELECT * FROM drinks WHERE Item = 'Bottled water'
SQLResult: [('Bottled water', '5.00', None, None, 'Single size = 20 ounces')]
Answer:The price of Bottled water is $5.00 for a single size of 20 ounces.

Question: What drinks are available in a medium size?
SQLQuery: SELECT Item, Price_Medium FROM drinks WHERE Price_Medium IS NOT NULL
> Finished chain.

> Finished chain.

Observation: The price of Bottled water is $5.00 for a single size of 20 ounces.

Question: What drinks are available in a medium size?
SQLQuery: SELECT Item, Price_Medium FROM drinks WHERE Price_Medium IS NOT NULL
Thought:Do I need to use a tool? Yes
Action: Menu Search
Action Input: drinks medium

> Entering new  chain...
Table names to use:
['drinks']

> Entering new  chain...
drinks medium
SQLQuery:

'Great! So to summarize your order, you would like a large pepperoni pizza with sausage and extra cheese for $17.95, and a single size of bottled water for $5.00. Is that correct? And would you like to pick up your order or have it delivered?'

In [20]:
agent_chain.run(input="This will be for pickup.")



> Entering new  chain...
Thought: Do I need to use a tool? No
AI: Great, your order will be for pickup. Is there anything else you would like to add to your order?

> Finished chain.


'Great, your order will be for pickup. Is there anything else you would like to add to your order?'

In [21]:
agent_chain.run(input="Please show me an itemized order summary to make sure it is right.")



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Order Summary
Action Input: "large pepperoni pizza with sausage and extra cheese for $17.95, and a single size of bottled water for $5.00"

> Entering new  chain...
Prompt after formatting:
This is a conversation between a human and a pizza order bot:

[HumanMessage(content='hi, i am Tim and my email is tweinzirl@gmail.com', additional_kwargs={}, example=False), AIMessage(content='Hi Tim! Welcome to our pizza restaurant. How can I assist you today?', additional_kwargs={}, example=False), HumanMessage(content='What is my name and email', additional_kwargs={}, example=False), AIMessage(content='Your name is Tim and your email is tweinzirl@gmail.com. How can I assist you with your order today?', additional_kwargs={}, example=False), HumanMessage(content='What is your most popular pizza?', additional_kwargs={}, example=False), AIMessage(content='Our most popular pizza is the Pepperoni pizza. Would you like to order on

'Order Summary:\n<br>Large Pepperoni Pizza with Sausage and Extra Cheese $17.95\n<br>Single Size Bottled Water $5.00\n<br>Total: $22.95'

In [22]:
agent_chain.run(input="That is it. Please email me my receipt.")



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Send Email
Action Input: "Large Pepperoni Pizza with Sausage and Extra Cheese for $17.95, Single Size Bottled Water for $5.00, Total: $22.95|tweinzirl@gmail.com"sending mail from OrderBotPizza.com to tweinzirl@gmail.com
Content-Type: multipart/mixed; boundary="===============5214729403428846177=="
MIME-Version: 1.0
From: OrderBotPizza.com
To: tweinzirl@gmail.com
Subject: Your OrderBot Pizza Order

--===============5214729403428846177==
Content-Type: text/html; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

Large Pepperoni Pizza with Sausage and Extra Cheese for $17.95, Single Size Bottled Water for $5.00, Total: $22.95
--===============5214729403428846177==--


Observation: (<email.mime.multipart.MIMEMultipart object at 0x7f4997676750>, 'mail sent successfully')
Thought:Do I need to use a tool? No
AI: Thank you for your order, Tim! Your receipt has been emailed to you. Is there anything else

'Thank you for your order, Tim! Your receipt has been emailed to you. Is there anything else I can assist you with today?'

In [23]:
for m in agent_chain.memory.chat_memory.messages:
    c = m.content.replace('$', '```$```')
    if m.type=="ai":
        display(Markdown(f'<span style="color: #ff0000">{m.type.upper()}</span>: {c}'))
    else:
        display(Markdown(f'<span style="color: cyan">{m.type.upper()}</span>: {c}'))

<span style="color: cyan">HUMAN</span>: hi, i am Tim and my email is tweinzirl@gmail.com

<span style="color: #ff0000">AI</span>: Hi Tim! Welcome to our pizza restaurant. How can I assist you today?

<span style="color: cyan">HUMAN</span>: What is my name and email

<span style="color: #ff0000">AI</span>: Your name is Tim and your email is tweinzirl@gmail.com. How can I assist you with your order today?

<span style="color: cyan">HUMAN</span>: What is your most popular pizza?

...

<span style="color: #ff0000">AI</span>: Great, your order will be for pickup. Is there anything else you would like to add to your order?

<span style="color: cyan">HUMAN</span>: Please show me an itemized order summary to make sure it is right.

<span style="color: #ff0000">AI</span>: Order Summary:
<br>Large Pepperoni Pizza with Sausage and Extra Cheese ```$```17.95
<br>Single Size Bottled Water ```$```5.00
<br>Total: ```$```22.95

<span style="color: cyan">HUMAN</span>: That is it. Please email me my receipt.

<span style="color: #ff0000">AI</span>: Thank you for your order, Tim! Your receipt has been emailed to you. Is there anything else I can assist you with today?